In [18]:
import numpy as np 
import pandas as pd
import os,re

In [4]:
df = pd.read_excel("/home/bruce1996/data/MCI/metadata/TPMIC_Diagnosis_297_KCF_0704.xlsx")
ab_matrix = pd.read_csv("/home/bruce1996/data/MCI/metaphlan_result/metaphlan_format_table.txt",sep='\t',index_col=0)

In [10]:
mci_sample = df['ID'][np.where((df['Diagnosis'] == 'MCI') | (df['Diagnosis'] == 'Dementia'),True,False)]
normal_sample = df['ID'][np.where(df['Diagnosis'] == 'Normal',True,False)]

In [14]:
mci_matrix = ab_matrix.loc[:,mci_sample]
normal_matrix = ab_matrix.loc[:,normal_sample]

In [26]:
p = '/home/bruce1996/data/MCI/manuscript_v3.2/'
output_path = {'mci' : p + 'mci/abundance_matrix/','normal' : p + 'normal/abundance_matrix/'}
bifido_idx = [bool(re.search('Bifidobacterium',x)) for x in ab_matrix.index]
lacto_idx = [bool(re.search('Lactobacillus',x)) for x in ab_matrix.index]

for key,output_dir in output_path.items() :
    if key == 'mci' :
        if os.path.isdir(output_dir) == False :
                os.mkdir(output_dir)
        ab_matrix.loc[:,mci_sample].to_csv(output_dir + 'mci_abundance_matrix.txt',sep='\t')
        ab_matrix.loc[bifido_idx,mci_sample].to_csv(output_dir + 'mci_bifido_abundance_matrix.txt',sep='\t')
        ab_matrix.loc[lacto_idx,mci_sample].to_csv(output_dir + 'mci_lacto_abundance_matrix.txt',sep='\t')
    else :
        if os.path.isdir(output_dir) == False :
                os.mkdir(output_dir)
        ab_matrix.loc[:,normal_sample].to_csv(output_dir + 'normal_abundance_matrix.txt',sep='\t')
        ab_matrix.loc[bifido_idx,normal_sample].to_csv(output_dir + 'normal_bifido_abundance_matrix.txt',sep='\t')
        ab_matrix.loc[lacto_idx,normal_sample].to_csv(output_dir + 'normal_lacto_abundance_matrix.txt',sep='\t')